# Flux computation

In [1]:
import pyvista
import lfricflux
import mint
import vtk
import numpy as np

In [2]:
# the LFRic diagnostic file containing W2h data
inputFile = '../data/gungho/original/lfric_diag.nc'
# configuration file that lists the vraiables of interest
configFile = '../configs/lfric.cfg'
# create a flux instance. The geometry is read from the netcdf Ugrid file
lf = lfricflux.LFRicFlux(configFile=configFile, inputFile=inputFile)

In [3]:
# compute the flux across an irregular line.
# The points are connected by straight lines in lon-lat space. To close the line, 
# let the last point be the same as the first. 
xy = [(-180., -60.), (180., 80.), (-70, 30), (-180., -60.)]
fluxes = lf.computeFlow(xy)

In [4]:
# the returned flux array has dimensions (num_time, num_elev)
print(fluxes)

[[  6.64218321  19.62590894  32.23001215  37.23704901  36.09820124
   26.4747387    9.69540677 -11.55785689 -34.21141811 -54.75861981
  -70.31887364 -79.14482736 -80.89079272 -76.40133271 -67.298658
  -55.51252629 -42.89761034 -30.95737131 -20.64473117 -12.35556127
   -6.13990427  -1.7689123    1.11717461   2.88675427   3.85605705
    4.26831023   4.30525031   4.09567618   3.73682895   3.24891983]]


In [5]:
# save the grid to file
lf.grid.dump('grid.vtk')

In [6]:
# instantiate the VTK grid object from file
reader = vtk.vtkUnstructuredGridReader()
reader.SetFileName('grid.vtk')
reader.Update() # read

In [7]:
# build the VTK grid 
grid = reader.GetOutput()
grid

<vtkmodules.vtkCommonDataModel.vtkUnstructuredGrid(0x7fa83f72a4d0) at 0x17d3250c0>

In [11]:
# plot
lf.buildFluxSurface(extrusion=10, cartesian=False, radius=1.0)
pyvista.global_theme.cmap = 'jet'
p = pyvista.Plotter(window_size=(900, 800))
p.set_background((0.9, 0.9, 0.92))
p.add_mesh(lf.targetGrid, scalars='flux', clim=(fluxes.min(), fluxes.max()))
p.add_mesh(grid, show_edges=True)
p.show()

Widget(value="<iframe src='http://localhost:60404/index.html?ui=P_0x188d9c940_2&reconnect=auto' style='width: …

In [9]:
from ipywidgets import interact
import ipywidgets as widgets

def callbackElev(k):
    # copy the new values
    lf.setSlice((0, k))
    lf.targetFlux.Modified()

nelevs = lf.getExtraDims()[-1]
interact(callbackElev, k=widgets.IntSlider(min=0, max=nelevs, step=1, value=0))

interactive(children=(IntSlider(value=0, description='k', max=30), Output()), _dom_classes=('widget-interact',…

<function __main__.callbackElev(k)>